In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


### Data Load

In [ ]:
movies = pd.read_csv('/gdrive/MyDrive/playdata/movie_recommendation/data/movies.csv')
movies.head(2)

,movie_id,title,release_date,imdb_url,all_genres
0,0,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,Animation-Children-Comedy
1,1,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,Action-Adventure-Thriller


In [ ]:
users = pd.read_csv('/gdrive/MyDrive/playdata/movie_recommendation/data/users.csv')
users.head(2)

,user_id,age,sex,occupation
0,0,24,M,technician
1,1,53,F,other


In [ ]:
ratings = pd.read_csv('/gdrive/MyDrive/playdata/movie_recommendation/data/ratings.csv')
ratings.head(2)

,user_id,movie_id,rating
0,195,241,3.0
1,185,301,3.0


- train, test set

In [ ]:
# df를 train/test set으로 분할하는 함수 생성
# Args: df, holdoyt_fraction(분할 비율)
def split_dataframe(df, holdout_fraction=0.1):
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]

  return train, test

In [ ]:
ratings['user_id'] = ratings['user_id'].astype(int)
ratings['movie_id'] = ratings['movie_id'].astype(int)

train_ratings, test_ratings = split_dataframe(ratings)
train_ratings.shape, test_ratings.shape

((90000, 3), (10000, 3))

### Matrix Factorization

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, dot

- Model 생성

In [ ]:
users.shape[0], movies.shape[0]

(943, 1682)

In [ ]:
n_latent_factors = 30 # user와 movie embedding의 차원수

user_input = Input(shape=[1], name='user')
movie_input = Input(shape=[1], name='movie')

user_embedding = Embedding(input_dim=users.shape[0] # 943
                           , output_dim = n_latent_factors # 30
                           , name='user_embedding'
                           )(user_input)

movie_embedding = Embedding(input_dim=movies.shape[0] # 1682
                           , output_dim = n_latent_factors # 30
                           , name='movie_embedding'
                           )(movie_input)

user_vec = Flatten(name='flatten_users')(user_embedding) # 1차원 배열로 변환
movie_vec = Flatten(name='flatten_movies')(movie_embedding) # 1차원 배열로 변환

product = dot([movie_vec, user_vec], axes=1) # 평점
model = Model(inputs=[user_input, movie_input], outputs=product) # user와 movie가 주어졌을 때, 평점을 예측하는 모델

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 movie (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 user (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 movie_embedding (Embedding)    (None, 1, 30)        50460       ['movie[0][0]']                  
                                                                                                  
 user_embedding (Embedding)     (None, 1, 30)        28290       ['user[0][0]']                   
                                                                                              

- compile

In [ ]:
model.compile(optimizer='adam', loss='mse')

- fit

In [ ]:
history = model.fit(x=[train_ratings['user_id'], train_ratings['movie_id']],
                    y=train_ratings['rating'], epochs=500,
                    validation_data=([test_ratings['user_id'], test_ratings['movie_id'] ],
                                     test_ratings['rating']),
                    verbose=1, batch_size=train_ratings.shape[0])

Epoch 1/500
1/1 [==============================] - 6s 6s/step - loss: 13.7204 - val_loss: 13.7865
Epoch 2/500
1/1 [==============================] - 0s 40ms/step - loss: 13.7195 - val_loss: 13.7865
Epoch 3/500
1/1 [==============================] - 0s 37ms/step - loss: 13.7186 - val_loss: 13.7865
Epoch 4/500
1/1 [==============================] - 0s 37ms/step - loss: 13.7177 - val_loss: 13.7865
Epoch 5/500
1/1 [==============================] - 0s 37ms/step - loss: 13.7168 - val_loss: 13.7864
Epoch 6/500
1/1 [==============================] - 0s 36ms/step - loss: 13.7157 - val_loss: 13.7862
Epoch 7/500
1/1 [==============================] - 0s 37ms/step - loss: 13.7146 - val_loss: 13.7858
Epoch 8/500
1/1 [==============================] - 0s 37ms/step - loss: 13.7133 - val_loss: 13.7852
Epoch 9/500
1/1 [==============================] - 0s 36ms/step - loss: 13.7119 - val_loss: 13.7845
Epoch 10/500
1/1 [==============================] - 0s 41ms/step - loss: 13.7103 - val_loss: 13.7836
E

In [ ]:
#@title Solution
DOT = 'dot'
COSINE = 'cosine'
def compute_scores(query_embedding, item_embeddings, measure=DOT):
  """Computes the scores of the candidates given a query.
  Args:
    query_embedding: a vector of shape [k], representing the query embedding.
    item_embeddings: a matrix of shape [N, k], such that row i is the embedding
      of item i.
    measure: a string specifying the similarity measure to be used. Can be
      either DOT or COSINE.
  Returns:
    scores: a vector of shape [N], such that scores[i] is the score of item i.
  """
  u = query_embedding
  V = item_embeddings
  if measure == COSINE:
    V = V / np.linalg.norm(V, axis=1, keepdims=True)
    u = u / np.linalg.norm(u)
  scores = u.dot(V.T)
  return scores

In [ ]:
# @title User recommendations and nearest neighbors (run this cell)
def user_recommendations(model, measure=DOT, exclude_rated=False, k=6, user_id=570):
  if USER_RATINGS:
    scores = compute_scores(
        model.embeddings["user_id"][user_id], model.embeddings["movie_id"], measure)
    score_key = measure + ' score'
    df = pd.DataFrame({
        score_key: list(scores),
        'movie_id': movies['movie_id'],
        'titles': movies['title'],
        'genres': movies['all_genres'],
    })
    if exclude_rated:
      # remove movies that are already rated
      rated_movies = ratings[ratings.user_id == str(user_id)]["movie_id"].values
      df = df[df.movie_id.apply(lambda movie_id: movie_id not in rated_movies)]
    return (df.sort_values([score_key], ascending=False).head(k))

In [ ]:
model.embeddings = {
    'user_id': model.get_layer('user_embedding').weights[0].numpy(), # U (943, 30)
    'movie_id': model.get_layer('movie_embedding').weights[0].numpy() # V (1682, 30)
}

In [ ]:
model.embeddings

{'user_id': array([[ 0.40883937,  0.33570138, -0.35649008, ..., -0.3966015 ,
          0.37839207,  0.36910602],
        [ 0.36142173,  0.34700963, -0.38880947, ..., -0.33601636,
          0.34684536,  0.33887446],
        [ 0.31100744,  0.3054987 , -0.33195472, ..., -0.25904152,
          0.34317115,  0.30425695],
        ...,
        [ 0.35957408,  0.3711364 , -0.37792096, ..., -0.43124118,
          0.39038998,  0.38934124],
        [ 0.43367326,  0.43012747, -0.35670826, ..., -0.3442395 ,
          0.443437  ,  0.38096777],
        [ 0.36218944,  0.3055697 , -0.37557966, ..., -0.36602095,
          0.3331448 ,  0.3641469 ]], dtype=float32),
 'movie_id': array([[ 0.40014994,  0.37585875, -0.40530464, ..., -0.3858588 ,
          0.323323  ,  0.3951084 ],
        [ 0.3345644 ,  0.34934676, -0.27182072, ..., -0.3193516 ,
          0.3629583 ,  0.34735832],
        [ 0.31729344,  0.30968627, -0.31615677, ..., -0.26176503,
          0.24370867,  0.27014643],
        ...,
        [ 0.2707

In [ ]:
# import pickle
# with open('CFModel_01.pickle','wb') as fw:
#     pickle.dump(model, fw)

In [ ]:
# with open('CFModel_01.pickle', 'rb') as f:
#     CFModel_01 = pickle.load(f)

In [ ]:
model.embeddings = {
    'user_id': model.get_layer('user_embedding').weights[0].numpy(), # U (943, 30)
    'movie_id': model.get_layer('movie_embedding').weights[0].numpy() # V (1682, 30)
}

In [ ]:
USER_RATINGS = True

In [ ]:
result_df = user_recommendations(model, measure=DOT, k=10, user_id=600, exclude_rated=True)

In [ ]:
result_df

,dot score,movie_id,titles,genres
813,4.143117,813,"Great Day in Harlem, A (1994)",Documentary
1642,4.132014,1642,Angel Baby (1995),Drama
850,4.065010,850,Two or Three Things I Know About Her (1966),Drama
1448,4.060930,1448,Pather Panchali (1955),Drama
1466,4.048428,1466,"Saint of Fort Washington, The (1993)",Drama
1641,4.039691,1641,Some Mother's Son (1996),Drama
1598,4.010055,1598,Someone Else's America (1995),Drama
1499,4.001060,1499,Santa with Muscles (1996),Comedy
1397,3.959460,1397,Anna (1996),Drama
1305,3.959085,1305,Delta of Venus (1994),Drama


In [ ]:
model.save('CFModel01.h5')

In [ ]:
import tensorflow as tf
model.save('CFModel01.h5')
CFModel01 = tf.keras.models.load_model('CFModel01.h5')
CFModel01.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 movie (InputLayer)             [(None, 1)]          0           []                               
                                                                                                  
 user (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 movie_embedding (Embedding)    (None, 1, 30)        50460       ['movie[0][0]']                  
                                                                                                  
 user_embedding (Embedding)     (None, 1, 30)        28290       ['user[0][0]']                   
                                                                                              